In [42]:
import pandas as pd
import datetime as dt

url = "https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/lyrics_cooccured/lyric_cooccured_by_line.csv"
words = pd.read_csv(url, encoding="utf-8").iloc[:, 1:]
words


,genre,year,song_id,title,line,noun,predicate
0,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),끝내(P)
1,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,하루(N),돌아가(P)
2,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),끝내(P)
3,성인가요,1964.0,4083218.0,내일또 만납시다,0.0,일(N),돌아가(P)
4,성인가요,1964.0,4083218.0,내일또 만납시다,2.0,하늘(N),반짝이(P)
...,...,...,...,...,...,...,...
182946,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,그늘(N),받(P)
182947,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),가리(P)
182948,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,사랑(N),받(P)
182949,발라드,2020.0,32998018.0,힘든 건 사랑이 아니다,15.0,미안(N),가리(P)


In [43]:
years = list(range(1985, 2020))
edges_filtered = pd.DataFrame()
for year in years:
    words_in_year = words[words["year"] == year]
    edges_cache = {}

    for _, word in words_in_year.iterrows():
        edges_cache.update(
            {
                (word["noun"], word["predicate"]): {
                    "weight": edges_cache.get(
                        (word["noun"], word["predicate"]), {}
                    ).get("weight", 0)
                    + 1
                }
            }
        )
    edges = pd.DataFrame.from_dict(edges_cache, orient="index")
    edges = edges.loc[edges["weight"] >= edges.quantile(0.95).weight].reset_index()
    edges["year"] = dt.date(year=year, month=1, day=1).isoformat()
    edges.columns = ["Source", "Target", "Weight", "Year"]
    edges = edges[["Year", "Source", "Target", "Weight"]]
    edges.index.names = ["ids"]
    edges_filtered = pd.concat([edges_filtered, edges])

edges_filtered


,Year,Source,Target,Weight
ids,,,,
0,1985-01-01,가슴(N),그리(P),4
1,1985-01-01,가슴(N),남(P),5
2,1985-01-01,가슴(N),뜨겁(P),4
3,1985-01-01,가슴(N),작(P),6
4,1985-01-01,가슴(N),저리(P),4
...,...,...,...,...
146,2019-01-01,필요(N),없(P),7
147,2019-01-01,하루(N),보(P),6
148,2019-01-01,하루(N),없(P),10


In [44]:
edges_filtered.to_csv(
    f"../data/lyrics_cooccured/lyric_coocuured_weighted_edges_1985_2019.csv",
    encoding="utf-8-sig",
)
